In [2]:
import re

In [3]:
re.sub('[ispe ]', '', 'this is a pen')

'than'

In [4]:
text = "-a-"
#re.sub("-", "", text[0])
#print(text)

In [7]:
# [ -/:-@\[-~]
# [!-/:-@[-`{-~]

In [12]:
text = "'p-d"
#regex = re.compile(r'^[ -/:-@\[-~]|[ -/:-@\[-~]$')
regex = re.compile(r'^[!-/:-@[-`{-~]')
result = regex.sub('', text)
print(result)

p-d


In [21]:
text = "@p-d@"
#regex = re.compile(r'^[ -/:-@\[-~]|[ -/:-@\[-~]$')
regex = re.compile(r'^[!-/:-@[-`{-~]|[!-/:-@[-`{-~]$')
result = regex.sub('', text)
print(result)

p-d
